In [141]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
import string

In [2]:
data = pd.read_csv("data/twitter_sentiment_x_y_train.csv")

In [200]:
#data.head(5)

In [144]:
#data.shape

(10980, 12)

In [104]:
x_test=pd.read_csv("data/twitter_sentiment_x_test.csv")

# Data Cleaning Phase

In [110]:
tweets_test=[tweet for tweet in x_test['text']] #created list of tweets for testing data
tweets_test_tokenized=[tweet.split()for tweet in tweets_test] # splitted into words
print(tweets_test_tokenized[0]) # just checking

['@AmericanAir', 'In', 'car', 'gng', 'to', 'DFW.', 'Pulled', 'over', '1hr', 'ago', '-', 'very', 'icy', 'roads.', 'On-hold', 'with', 'AA', 'since', '1hr.', "Can't", 'reach', 'arpt', 'for', 'AA2450.', 'Wat', '2', 'do?']


In [26]:
# reforming training data

In [85]:
tweets_train=[tweet for tweet in data['text']] #created list of tweets for training data
sentiments_train=[sentiment for sentiment in data['airline_sentiment']] # created list of y_class (sentiment)
tweets_tokenized=[tweet.split()for tweet in tweets_train] # splitted into words

#NLTK

In [91]:
lemmatizer = WordNetLemmatizer() # object created of WordNetLemmatizer

Simple Function to get pos_tags that can be passed as a parameter to lemmatizer

In [90]:
def get_simple_pos(tag):
    
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

Stop Words and Punctuations (to be removed)

In [93]:
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)
stops.update(punctuations)

Function to clean the words of every tweet using  lemmatizer

In [94]:
def clean(words):
    output_words = []
    for w in words:
        if w.lower() not in stops:
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

Cleaned and stored the tweets of training in tweets_tokenized

In [ ]:
tweets_tokenized=[clean(tweet) for tweet in tweets_tokenized]
#dont run currently

Cleaned and stored the tweets of testing in tweets_test_tokenized
Also noted the time.

In [111]:
import time
start=time.time()
tweets_test_tokenized=[clean(tweet) for tweet in tweets_test_tokenized]
#works fine
print(start-time.time())

-695.8841910362244


In [97]:
print(tweets_tokenized[0])

['@southwestair', 'schedule', 'morning,', '2', 'day', 'fact,', 'yes..not', 'sure', 'even', 'flight', 'one', 'cancelled', 'flightled']


Reformatted to be passed into Countvectorizer

In [201]:
tweets_train_text_format=[" ".join(tweet) for tweet in tweets_tokenized]
tweets_test_text_format=[" ".join(tweet) for tweet in tweets_test_tokenized]
#tweets_test_tokenized

Passed  to CountVectorizer with optimal parameters to convert into sklearn_oriented x_train,y_train sparses matrix format

In [263]:
count_vec = CountVectorizer(max_features = 2000,ngram_range=(1, 3),max_df=0.8)
x_train_features = count_vec.fit_transform(tweets_train_text_format)
print(x_train_features.todense())

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 1 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [247]:
print(count_vec.get_feature_names())

['000', '10', '10 min', '100', '11', '12', '12 hour', '13', '14', '15', '16', '17', '18', '19', '1hr', '1k', '1st', '1st class', '20', '20 min', '20 minute', '200', '2015', '21', '22', '23', '24', '24 hour', '24hrs', '25', '25 min', '26', '27', '2hrs', '2nd', '2nd time', '2x', '30', '30 min', '30 minute', '35', '36', '3rd', '40', '40 min', '40 minute', '400', '45', '45 min', '45 minute', '50', '500', '60', '70', '700', '728', '75', '7am', '800', '800 number', '90', 'aa', 'able', 'able get', 'absolute', 'absolutely', 'absurd', 'accept', 'acceptable', 'access', 'accommodate', 'account', 'acct', 'act', 'actual', 'actually', 'add', 'additional', 'address', 'advantage', 'advise', 'advisory', 'afternoon', 'again', 'age', 'agent', 'agents', 'ago', 'ah', 'ahead', 'air', 'aircraft', 'airfare', 'airline', 'airline cancelled', 'airline ever', 'airlines', 'airplane', 'airport', 'airway', 'airways', 'alert', 'all', 'allow', 'almost', 'almost hour', 'alone', 'along', 'already', 'also', 'alternate', 

In [269]:
x_test_features = count_vec.transform(tweets_test_text_format)

# CLASSIFICATION PHASE

Using Random Forest Classifier

In [222]:
from sklearn.ensemble import RandomForestClassifier

In [223]:
rfc = RandomForestClassifier()
rfc.fit(x_train_features, sentiments)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

Finding the optimal parameters

In [234]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
grid_search_rf = GridSearchCV(estimator = rfc, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [236]:
grid_search_rf.fit(x_train_features, sentiments)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 10.2min
[Parallel(n_jobs=-1)]: Done 864 out of 864 | elapsed: 13.9min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'bootstrap': [True], 'max_depth': [80, 90, 100, 110], 'max_features': [2, 3], 'min_samples_leaf': [3, 4, 5], 'min_samples_split': [8, 10, 12], 'n_estimators': [100, 200, 300, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=2)

In [244]:
best_grid = grid_search_rf.best_estimator_
print(best_grid)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=110, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=12,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


Predictions by ensemble

In [ ]:
y_pred_rf=grid_search_rf.predict(x_test_features)

USING Support Vector Machine after Grid Search

In [266]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
abc = SVC()
grid = {'C' : [1e2, 1e3, 5e3, 1e4, 5e4],
       'gamma' : [1e-3, 5e-4, 1e-4]}
svc = GridSearchCV(abc, grid)
svc.fit(x_train_features, sentiments)

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [100.0, 1000.0, 5000.0, 10000.0, 50000.0], 'gamma': [0.001, 0.0005, 0.0001]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [249]:
print(svc.best_estimator_)

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [270]:
y_pred_sv=svc.predict(x_test_features)

Storing  the results in different csv files

In [271]:
np.savetxt('twitter_sentiment_pred_svm.csv', y_pred_sv , delimiter=" ", fmt="%s")

In [ ]:
np.savetxt('twitter_sentiment_pred_rf.csv', y_pred_rf , delimiter=" ", fmt="%s")

Random Forest came out to be really well, 0.76 accuracy without  grid-search but overfitted way too much after the grid search and ended up giving only 0.67 accuracy.
On the other hand svc gave 0.45 accuracy without grid search and 0.77 after grid search and cross validation.